In [ ]:
"""
SISTEM REKOMENDASI MUSIK BERBASIS MACHINE LEARNING
================================================
Sistem rekomendasi musik yang menggunakan content-based filtering dengan clustering
dan similarity analysis untuk memberikan rekomendasi lagu yang akurat berdasarkan
karakteristik audio dan preferensi pengguna.

Teknologi yang digunakan:
- K-Means Clustering untuk pengelompokan genre dan lagu
- t-SNE dan PCA untuk visualisasi dan dimensionality reduction  
- Cosine Similarity untuk perhitungan kesamaan antar lagu
- Spotify API untuk integrasi data real-time

"""

# Import library utama untuk analisis data
import os
import numpy as np
import pandas as pd

# Import library visualisasi data dan analisis statistik
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline

# Import library machine learning untuk clustering dan preprocessing
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

# Suppress warnings untuk output yang lebih bersih
import warnings
warnings.filterwarnings("ignore")

print("Semua library berhasil dimuat!")
print("Sistem Rekomendasi Musik siap digunakan")

In [ ]:
# Uncomment jika mount lewat Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
"""
LOADING DATA MUSIK
==================
Memuat dataset musik dari file CSV lokal yang berisi:
1. data.csv - Dataset utama lagu dengan fitur audio
2. data_by_genres.csv - Data agregat berdasarkan genre
3. data_by_year.csv - Data agregat berdasarkan tahun
"""

# Konfigurasi path file CSV (sesuaikan dengan lokasi file Anda)
data_path = 'data/data.csv'  # UBAH PATH INI sesuai lokasi file
genre_data_path = 'data/data_by_genres.csv'
year_data_path = 'data/data_by_year.csv'

try:
    # Load dataset utama musik dengan error handling
    print("Memuat dataset musik...")
    data = pd.read_csv(data_path)
    genre_data = pd.read_csv(genre_data_path)
    year_data = pd.read_csv(year_data_path)  # FIX: was year_data_path = 
    
    # Tampilkan informasi dataset
    print(f"Dataset berhasil dimuat!")
    print(f"# Jumlah lagu: {len(data):,}")
    print(f"# Jumlah genre: {len(genre_data):,}")
    print(f"# Rentang tahun: {len(year_data):,}")
    
    # Display struktur data
    print("\nPreview data:")
    print(data.head())
    
except FileNotFoundError as e:
    print(f"Error: File tidak ditemukan - {e}")
    print("Pastikan path file sudah benar!")

In [ ]:
"""
CLUSTERING GENRE MUSIK
======================
Mengelompokkan genre musik berdasarkan karakteristik audio menggunakan K-Means
untuk mengidentifikasi pola dan kesamaan antar genre.
"""

# Konfigurasi parameter clustering
N_CLUSTERS_GENRE = 10
RANDOM_STATE = 42

print("Memulai clustering genre musik...")

# Membuat pipeline clustering dengan standardisasi dan K-Means
cluster_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Normalisasi fitur audio
    ('kmeans', KMeans(n_clusters=N_CLUSTERS_GENRE, random_state=RANDOM_STATE))
])

# Fit pipeline dan prediksi cluster untuk data genre
print(f"Melakukan clustering dengan {N_CLUSTERS_GENRE} cluster...")
numerical_features = genre_data.select_dtypes(include=[np.number])
genre_data['cluster'] = cluster_pipeline.fit_predict(numerical_features)

# Analisis hasil clustering
print("Clustering genre selesai!")
print(f"Distribusi cluster:")
print(genre_data['cluster'].value_counts().sort_index())

# Tampilkan contoh genre per cluster
print(f"\nSample genre per cluster:")
for cluster_id in sorted(genre_data['cluster'].unique()):
    genres_in_cluster = genre_data[genre_data['cluster'] == cluster_id]['genres'].head(3).tolist()
    print(f"  Cluster {cluster_id}: {', '.join(genres_in_cluster)}")

In [ ]:
"""
VISUALISASI t-SNE UNTUK GENRE
=============================
Menggunakan t-SNE untuk mereduksi dimensi data genre dan memvisualisasikan
pengelompokan genre dalam ruang 2D untuk analisis eksploratori.
"""

# Konfigurasi parameter t-SNE
TSNE_COMPONENTS = 2
TSNE_PERPLEXITY = 30
TSNE_RANDOM_STATE = 42

print("Memulai dimensionality reduction dengan t-SNE...")
print(f"Mereduksi dari {len(numerical_features.columns)} dimensi ke {TSNE_COMPONENTS} dimensi")

# Membuat pipeline t-SNE dengan standardisasi
tsne_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('tsne', TSNE(n_components=TSNE_COMPONENTS, 
                  perplexity=TSNE_PERPLEXITY,
                  random_state=TSNE_RANDOM_STATE,
                  verbose=1))
])

# Fit dan transform data genre
tsne_results = tsne_pipeline.fit_transform(numerical_features)

# Membuat DataFrame untuk visualisasi
projection = pd.DataFrame(tsne_results, columns=['x', 'y'])
projection['genres'] = genre_data['genres'].values
projection['cluster'] = genre_data['cluster'].values

print("t-SNE selesai! Membuat visualisasi...")

# Membuat scatter plot interaktif dengan Plotly
fig = px.scatter(
    projection, 
    x='x', y='y', 
    color='cluster',
    hover_data=['genres'],
    title='Visualisasi t-SNE: Pengelompokan Genre Musik',
    labels={'x': 't-SNE Komponen 1', 'y': 't-SNE Komponen 2'},
    color_continuous_scale='viridis'
)

fig.update_layout(
    width=800, height=600,
    showlegend=True
)

print("Menampilkan plot interaktif...")
fig.show()

In [ ]:
"""
CLUSTERING LAGU INDIVIDUAL
==========================
Mengelompokkan lagu-lagu berdasarkan fitur audio menggunakan K-Means clustering
untuk mengidentifikasi pola kesamaan musik dan membantu sistem rekomendasi.
"""

# Konfigurasi parameter clustering lagu
N_CLUSTERS_SONGS = 20
RANDOM_STATE = 42

print("Memulai clustering lagu berdasarkan fitur audio...")

# Membuat pipeline clustering untuk lagu dengan lebih banyak cluster
song_cluster_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Normalisasi fitur untuk performa clustering optimal
    ('kmeans', KMeans(n_clusters=N_CLUSTERS_SONGS, 
                      random_state=RANDOM_STATE,
                      max_iter=300,
                      n_init=10))
])

# Memilih fitur numerik dari dataset lagu
X = data.select_dtypes(include=[np.number])
number_cols = list(X.columns)

print(f"Fitur yang digunakan untuk clustering: {len(number_cols)} fitur")
print(f"Fitur: {', '.join(number_cols[:8])}..." if len(number_cols) > 8 else f"🔢 Fitur: {', '.join(number_cols)}")

# Melakukan clustering
print(f"Melakukan clustering {len(X):,} lagu ke dalam {N_CLUSTERS_SONGS} cluster...")
song_cluster_pipeline.fit(X)
song_cluster_labels = song_cluster_pipeline.predict(X)

# Menambahkan hasil clustering ke dataset
data['cluster_label'] = song_cluster_labels

# Analisis hasil clustering
print("Clustering lagu selesai!")
print(f"Distribusi lagu per cluster:")
cluster_counts = pd.Series(song_cluster_labels).value_counts().sort_index()
for cluster_id, count in cluster_counts.items():
    print(f"  Cluster {cluster_id}: {count:,} lagu ({count/len(data)*100:.1f}%)")

# Simpan scaler untuk digunakan dalam rekomendasi
print(f"Pipeline clustering tersimpan untuk sistem rekomendasi")

In [ ]:
"""
VISUALISASI PCA UNTUK LAGU
==========================
Menggunakan Principal Component Analysis (PCA) untuk mereduksi dimensi data lagu
dan memvisualisasikan distribusi cluster dalam ruang 2D.
"""

# Konfigurasi PCA
PCA_COMPONENTS = 2
RANDOM_STATE = 42

print("Memulai analisis PCA untuk visualisasi lagu...")
print(f"Mereduksi dari {len(X.columns)} dimensi ke {PCA_COMPONENTS} dimensi")

# Membuat pipeline PCA dengan standardisasi
pca_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=PCA_COMPONENTS, random_state=RANDOM_STATE))
])

# Fit dan transform data lagu
song_embedding = pca_pipeline.fit_transform(X)

# Analisis explained variance
pca_model = pca_pipeline.named_steps['pca']
explained_variance = pca_model.explained_variance_ratio_
total_variance = sum(explained_variance)

print(f"Explained Variance:")
print(f"  PC1: {explained_variance[0]:.3f} ({explained_variance[0]*100:.1f}%)")
print(f"  PC2: {explained_variance[1]:.3f} ({explained_variance[1]*100:.1f}%)")
print(f"  Total: {total_variance:.3f} ({total_variance*100:.1f}%)")

# Membuat DataFrame untuk visualisasi
projection = pd.DataFrame(song_embedding, columns=['PC1', 'PC2'])
projection['title'] = data['name'].values
projection['artists'] = data['artists'].values
projection['cluster'] = data['cluster_label'].values
projection['year'] = data['year'].values

print("PCA selesai! Membuat visualisasi...")

# Membuat scatter plot interaktif
fig = px.scatter(
    projection, 
    x='PC1', y='PC2', 
    color='cluster',
    hover_data=['title', 'artists', 'year'],
    title='📊 Visualisasi PCA: Distribusi Cluster Lagu',
    labels={'PC1': f'PC1 ({explained_variance[0]*100:.1f}%)', 
            'PC2': f'PC2 ({explained_variance[1]*100:.1f}%)'},
    color_continuous_scale='viridis'
)

fig.update_layout(
    width=900, height=600,
    showlegend=True
)

print("Menampilkan plot interaktif...")
fig.show()

print(f"💡 Interpretasi: PCA menjelaskan {total_variance*100:.1f}% variasi data")

In [ ]:
"""
SETUP SPOTIFY API (OPSIONAL)
============================
Instalasi library Spotify jika diperlukan untuk integrasi real-time.
Uncomment baris di bawah jika library belum terinstall.
"""

# !pip install spotipy
print("Spotify library sudah tersedia")
print("Jika error, uncomment baris pip install di atas")

In [ ]:
"""
INTEGRASI SPOTIFY API
====================
Konfigurasi koneksi ke Spotify API untuk mendapatkan data lagu real-time.
⚠️  SECURITY WARNING: Sebaiknya gunakan environment variables untuk API keys!
"""

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict

# ⚠️  GANTI DENGAN CREDENTIALS ANDA SENDIRI
# Sebaiknya gunakan environment variables: os.getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_ID = ''
SPOTIFY_CLIENT_SECRET = ''

print("🔐 Menginisialisasi koneksi Spotify API...")

try:
    # Inisialisasi client Spotify
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_CLIENT_SECRET
    ))
    print("Koneksi Spotify API berhasil!")
except Exception as e:
    print(f"Error koneksi Spotify: {e}")
    sp = None

def find_song(name, year):
    """
    Mencari detail lagu dari Spotify API berdasarkan nama dan tahun.
    
    Parameters:
    -----------
    name : str
        Nama lagu yang dicari
    year : int
        Tahun rilis lagu
        
    Returns:
    --------
    pd.DataFrame atau None
        DataFrame berisi detail lagu atau None jika tidak ditemukan
    """
    if sp is None:
        print("Spotify API tidak tersedia")
        return None
        
    try:
        print(f"Mencari lagu: '{name}' ({year})")
        
        # Query pencarian lagu
        query = f'track:"{name}" year:{year}'
        results = sp.search(q=query, limit=1, type='track')
        
        if not results['tracks']['items']:
            print(f"Lagu tidak ditemukan: {name} ({year})")
            return None
            
        # Ekstrak detail lagu
        track = results['tracks']['items'][0]
        track_id = track['id']
        
        # Dapatkan audio features
        audio_features = sp.audio_features(track_id)[0]
        
        if not audio_features:
            print(f"Audio features tidak tersedia untuk: {name}")
            return None
            
        # Bangun dictionary data lagu
        song_data = {
            'name': [name],
            'year': [year],
            'explicit': [int(track['explicit'])],
            'duration_ms': [track['duration_ms']],
            'popularity': [track['popularity']]
        }
        
        # Tambahkan audio features
        audio_feature_keys = ['acousticness', 'danceability', 'energy', 'instrumentalness',
                             'key', 'liveness', 'loudness', 'mode', 'speechiness', 
                             'tempo', 'valence']
        
        for key in audio_feature_keys:
            if key in audio_features and audio_features[key] is not None:
                song_data[key] = [audio_features[key]]
                
        print(f"Lagu ditemukan: {track['name']} - {track['artists'][0]['name']}")
        return pd.DataFrame(song_data)
        
    except Exception as e:
        print(f"Error mencari lagu: {e}")
        return None

print("Fungsi pencarian Spotify siap digunakan!")

In [ ]:
"""
SISTEM REKOMENDASI MUSIK UTAMA
==============================
Engine rekomendasi musik berbasis content-based filtering menggunakan cosine similarity
dan machine learning clustering untuk memberikan rekomendasi yang akurat.
"""

from collections import defaultdict
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
import difflib

# Definisi fitur audio yang digunakan untuk perhitungan similarity
AUDIO_FEATURES = [
    'valence', 'year', 'acousticness', 'danceability', 'duration_ms', 
    'energy', 'explicit', 'instrumentalness', 'key', 'liveness', 
    'loudness', 'mode', 'popularity', 'speechiness', 'tempo'
]

# Kolom metadata untuk output rekomendasi
METADATA_COLS = ['name', 'year', 'artists']

print("Inisialisasi Sistem Rekomendasi Musik")
print(f"Menggunakan {len(AUDIO_FEATURES)} fitur audio untuk analisis")

def get_song_data(song, spotify_data):
    """
    Mengambil data lagu dari dataset berdasarkan nama dan tahun.
    
    Parameters:
    -----------
    song : dict
        Dictionary berisi 'name' dan 'year' lagu
    spotify_data : pd.DataFrame
        Dataset musik untuk pencarian
        
    Returns:
    --------
    pd.Series atau None
        Data lagu jika ditemukan, None jika tidak ada
    """
    try:
        print(f"Mencari lagu: '{song['name']}' ({song['year']})")
        
        # Filter data berdasarkan nama dan tahun lagu
        filtered_data = spotify_data[
            (spotify_data['name'].str.lower() == song['name'].lower()) & 
            (spotify_data['year'] == song['year'])
        ]
        
        if not filtered_data.empty:
            song_data = filtered_data.iloc[0]
            print(f"Lagu ditemukan dalam dataset lokal")
            return song_data
        else:
            print(f"Lagu tidak ditemukan: {song['name']} ({song['year']})")
            # Fallback ke Spotify API jika tersedia
            return find_song(song['name'], song['year']) if 'find_song' in globals() else None
            
    except Exception as e:
        print(f"Error saat mencari lagu: {e}")
        return None

def get_mean_vector(song_list, spotify_data):
    """
    Menghitung vektor rata-rata dari daftar lagu untuk representasi preferensi pengguna.
    
    Parameters:
    -----------
    song_list : list
        List berisi dictionary lagu dengan 'name' dan 'year'
    spotify_data : pd.DataFrame
        Dataset musik
        
    Returns:
    --------
    np.array
        Vektor rata-rata fitur audio dari lagu-lagu input
    """
    print(f"Menghitung vektor rata-rata dari {len(song_list)} lagu...")
    song_vectors = []
    found_songs = 0
    
    for i, song in enumerate(song_list, 1):
        print(f"  ({i}/{len(song_list)}) Memproses: {song['name']}")
        song_data = get_song_data(song, spotify_data)
        
        if song_data is not None:
            try:
                # Ambil fitur audio numerik
                song_vector = song_data[AUDIO_FEATURES].values
                song_vectors.append(song_vector)
                found_songs += 1
            except KeyError as e:
                print(f"    Fitur tidak lengkap: {e}")
        else:
            print(f"    Lagu tidak dapat diproses")
    
    if not song_vectors:
        raise ValueError("Tidak ada lagu yang dapat diproses untuk rekomendasi")
    
    print(f"Berhasil memproses {found_songs}/{len(song_list)} lagu")
    song_matrix = np.array(song_vectors)
    mean_vector = np.mean(song_matrix, axis=0)
    
    return mean_vector

def flatten_dict_list(dict_list):
    """
    Meratakan list dictionary untuk filtering duplikasi.
    
    Parameters:
    -----------
    dict_list : list
        List berisi dictionary lagu
        
    Returns:
    --------
    defaultdict
        Dictionary yang diratakan dengan key-value lists
    """
    flattened_dict = defaultdict(list)
    
    for dictionary in dict_list:
        for key, value in dictionary.items():
            flattened_dict[key].append(value)
            
    return flattened_dict

def recommend_songs(song_list, spotify_data, n_songs=10):
    """
    Fungsi utama untuk memberikan rekomendasi lagu berdasarkan content-based filtering.
    
    Parameters:
    -----------
    song_list : list
        List lagu input dari pengguna
    spotify_data : pd.DataFrame
        Dataset musik untuk pencarian kandidat
    n_songs : int, default=10
        Jumlah rekomendasi yang diinginkan
        
    Returns:
    --------
    list
        List dictionary berisi rekomendasi lagu
    """
    print("MEMULAI PROSES REKOMENDASI")
    print("=" * 50)
    
    try:
        # Langkah 1: Validasi input
        if not song_list or len(song_list) == 0:
            raise ValueError("Input lagu tidak boleh kosong")
            
        print(f"Input: {len(song_list)} lagu untuk analisis preferensi")
        print(f"Target: {n_songs} rekomendasi lagu")
        
        # Langkah 2: Hitung vektor preferensi pengguna
        song_center = get_mean_vector(song_list, spotify_data)
        
        # Langkah 3: Siapkan data untuk perhitungan similarity
        print("Mempersiapkan perhitungan similarity...")
        
        # Pastikan semua fitur tersedia dalam dataset
        available_features = [col for col in AUDIO_FEATURES if col in spotify_data.columns]
        if len(available_features) != len(AUDIO_FEATURES):
            print(f"Hanya {len(available_features)}/{len(AUDIO_FEATURES)} fitur tersedia")
        
        # Ambil scaler dari pipeline clustering yang sudah dilatih
        scaler = song_cluster_pipeline.steps[0][1]
        
        # Normalisasi data
        scaled_data = scaler.transform(spotify_data[available_features])
        scaled_song_center = scaler.transform(song_center[:len(available_features)].reshape(1, -1))
        
        # Langkah 4: Hitung cosine similarity
        print("Menghitung cosine similarity...")
        distances = cdist(scaled_song_center, scaled_data, 'cosine')
        
        # Langkah 5: Ambil kandidat terbaik
        candidate_indices = np.argsort(distances[0])[:n_songs*2]  # Ambil 2x lebih banyak untuk filtering
        candidate_songs = spotify_data.iloc[candidate_indices].copy()
        
        # Langkah 6: Filter lagu yang sudah ada dalam input
        song_dict = flatten_dict_list(song_list)
        input_song_names = [name.lower() for name in song_dict['name']]
        
        filtered_songs = candidate_songs[
            ~candidate_songs['name'].str.lower().isin(input_song_names)
        ]
        
        # Langkah 7: Ambil top N rekomendasi
        final_recommendations = filtered_songs.head(n_songs)
        
        # Langkah 8: Format output
        recommendations = []
        for idx, song in final_recommendations.iterrows():
            rec = {}
            for col in METADATA_COLS:
                if col in song:
                    rec[col] = song[col]
            
            # Tambahkan similarity score
            orig_idx = candidate_songs.index.get_loc(idx)
            similarity_score = 1 - distances[0][candidate_indices[orig_idx]]
            rec['similarity_score'] = round(similarity_score, 4)
            
            recommendations.append(rec)
        
        # Langkah 9: Tampilkan hasil
        print("REKOMENDASI BERHASIL DIBUAT")
        print("=" * 50)
        print(f"Ditemukan {len(recommendations)} rekomendasi:")
        
        for i, rec in enumerate(recommendations, 1):
            artist_info = f" - {rec.get('artists', 'Unknown')}" if 'artists' in rec else ""
            similarity = rec.get('similarity_score', 0)
            print(f"  {i:2d}. {rec['name']} ({rec.get('year', 'N/A')}){artist_info}")
            print(f"      Similarity: {similarity:.3f}")
        
        return recommendations
        
    except Exception as e:
        print(f"ERROR dalam proses rekomendasi: {e}")
        return []

print("Sistem Rekomendasi Musik siap digunakan!")
print("Gunakan: recommend_songs([{'name': 'Nama Lagu', 'year': 2020}], data)")

In [ ]:
"""
DEMO & TESTING SISTEM REKOMENDASI
=================================
Menguji sistem rekomendasi dengan beberapa contoh lagu untuk memvalidasi
kualitas dan akurasi rekomendasi yang dihasilkan.
"""

print("MEMULAI DEMO SISTEM REKOMENDASI MUSIK")
print("=" * 60)

# Test Case 1: Single Song Recommendation
print("\nTEST CASE 1: Rekomendasi berdasarkan 1 lagu")
print("-" * 40)

test_song_1 = [{'name': 'Basket Case', 'year': 1994}]
print(f"Input: Green Day - Basket Case (1994)")
print("Mencari lagu serupa (punk rock/alternative)...")

try:
    recommendations_1 = recommend_songs(test_song_1, data, n_songs=5)
    
    if recommendations_1:
        print(f"\nBerhasil! Ditemukan {len(recommendations_1)} rekomendasi terbaik")
    else:
        print("Tidak ada rekomendasi yang ditemukan")
        
except Exception as e:
    print(f"Error: {e}")

# Test Case 2: Multiple Songs Recommendation  
print("\n\nTEST CASE 2: Rekomendasi berdasarkan multiple lagu")
print("-" * 50)

test_songs_2 = [
    {'name': 'Bohemian Rhapsody', 'year': 1975},
    {'name': 'Stairway to Heaven', 'year': 1971},
    {'name': 'Hotel California', 'year': 1976}
]

print("Input: Classic Rock Playlist")
for song in test_songs_2:
    print(f"  • {song['name']} ({song['year']})")
print("Mencari lagu classic rock serupa...")

try:
    recommendations_2 = recommend_songs(test_songs_2, data, n_songs=8)
    
    if recommendations_2:
        print(f"\nBerhasil! Ditemukan {len(recommendations_2)} rekomendasi")
        
        # Analisis genre diversity
        if len(recommendations_2) > 0:
            years = [rec.get('year', 0) for rec in recommendations_2 if rec.get('year')]
            if years:
                avg_year = sum(years) / len(years)
                year_range = max(years) - min(years)
                print(f"📊 Analisis Rekomendasi:")
                print(f"  • Rata-rata tahun: {avg_year:.0f}")
                print(f"  • Rentang tahun: {year_range} tahun")
    else:
        print("Tidak ada rekomendasi yang ditemukan")
        
except Exception as e:
    print(f"Error: {e}")

# Test Case 3: Modern Pop Music
print("\n\nTEST CASE 3: Rekomendasi musik modern")
print("-" * 40)

test_songs_3 = [
    {'name': 'Blinding Lights', 'year': 2019},
    {'name': 'Watermelon Sugar', 'year': 2020}
]

print("🎤 Input: Modern Pop Hits")
for song in test_songs_3:
    print(f"  • {song['name']} ({song['year']})")
print("Mencari lagu pop modern serupa...")

try:
    recommendations_3 = recommend_songs(test_songs_3, data, n_songs=6)
    
    if recommendations_3:
        print(f"\nBerhasil! Ditemukan {len(recommendations_3)} rekomendasi")
    else:
        print("Tidak ada rekomendasi yang ditemukan")
        
except Exception as e:
    print(f"Error: {e}")

# Performance Summary
print("\n\nRINGKASAN PERFORMA SISTEM")
print("=" * 40)
print("Sistem Rekomendasi Musik telah diuji dengan 3 test case")
print("Menggunakan Content-Based Filtering dengan Cosine Similarity")
print("Fitur: Audio features, tahun, popularitas, dan karakteristik musik")
print("Pipeline: Standardisasi → K-Means Clustering → Similarity Calculation")

print("\nCARA PENGGUNAAN:")
print("recommend_songs([{'name': 'Nama Lagu', 'year': Tahun}], data, n_songs=10)")

print("\nDemo selesai! Sistem siap untuk penggunaan production.")